In [31]:
from mesa import Model
from mesa import Agent
from mesa.space import MultiGrid
from mesa.time import RandomActivation
from mesa.datacollection import DataCollector
import random

In [32]:
class AmongUs(Model):
    def __init__(self, map_layout, n_crew, n_impo):
        super().__init__()

        # generate grid
        # self.height = 114
        # self.width = 114
        self.height = 60
        self.width = 60
        self.grid = MultiGrid(self.width, self.height, torus=True)

        # generate map
        self.generate_map(map_layout)

        # create schedulers for agents
        self.schedule_Crewmate = RandomActivation(self)
        self.schedule_Imposter = RandomActivation(self)
        
        # initialize agents
        self.n_crew = n_crew
        self.n_impo = n_impo
        self.spawn_players()

    def generate_map(self, map_layout):
        map_layout = [
            (1,1),
            (2,1),
            (3,1),
            (4,1),
            (5,1),
            (6,1),
            (7,1),
            (8,1),
            (9,1),
            (10,1),
            (1,1),
            (1,2),
            (1,3),
            (1,4),
            (1,5),
            (1,6),
            (1,7),
            (1,8),
            (1,9),
            (1,10),
            (2,10),
            (3,10),
            (4,10),
            (5,10),
            (6,10),
            (7,10),
            (8,10),
            (9,10),
            (10,10),
            (10,1),
            (10,2),
            (10,3),
            (10,4),
            (10,5),
            (10,6),
            (10,7),
            (10,8),
            (10,9)]


        for coord in map_layout:
            self.new_agent(Wall, coord)

    def spawn_players(self):
        # generate crewmates
        for _ in range(self.n_crew):
            self.new_agent(Crewmate, (5, 5))

        # generate imposters
        for _ in range(self.n_impo):
            self.new_agent(Imposter, (5, 5))

    def new_agent(self, agent_type, pos):
        '''
        Method that creates a new agent, and adds it to the correct scheduler.
        '''
        agent = agent_type(self.next_id(), self, pos)

        self.grid.place_agent(agent, pos)

        if agent_type == Crewmate or agent_type == Imposter:
            getattr(self, f'schedule_{agent_type.__name__}').add(agent)

        
    def step(self):
        '''
        Method that steps every agent. 
        
        Prevents applying step on new agents by creating a local list.
        '''

        self.schedule_Crewmate.step()
        self.schedule_Imposter.step()

        # self.datacollector.collect(self)

    def play_match(self, step_count=50):
        '''
        Method that runs the model for a specific amount of steps.
        '''
        for i in range(step_count):
            self.step()


In [33]:
class Crewmate(Agent):
    def __init__(self, unique_id, model, init_pos):
        super().__init__(unique_id, model)
        self.pos = init_pos

    def step(self):
        grid = self.model.grid
        neighborhood = grid.get_neighborhood(self.pos, True)

        neighbors = self.model.grid.get_neighbors(self.pos, True)

        if len(neighbors) > 0:
            for neighbor in neighbors:
                if type(neighbor) == Wall and neighbor.pos in neighborhood:
                    neighborhood.remove(neighbor.pos)

        grid.move_agent(self, random.choice(neighborhood))

class Imposter(Agent):
    def __init__(self, unique_id, model, init_pos):
        super().__init__(unique_id, model)
        self.pos = init_pos

    def step(self):
        grid = self.model.grid
        neighborhood = grid.get_neighborhood(self.pos, True)

        neighbors = self.model.grid.get_neighbors(self.pos, True)

        if len(neighbors) > 0:
            for neighbor in neighbors:
                if type(neighbor) == Wall and neighbor.pos in neighborhood:
                    neighborhood.remove(neighbor.pos)

        grid.move_agent(self, random.choice(neighborhood))

class Wall(Agent):
    def __init__(self, unique_id, model, pos):
        super().__init__(unique_id, model)
        self.pos = pos

In [34]:
amongUs = AmongUs(0, 3, 1)

amongUs.play_match()